OK I am thinking about consolidating the py/not types. So I need to figure out how boxing/unboxing works.

Basically, I wanna use the same types internally for the py/not types. So I can use the same numba ttypes in kernels and not.

But that means I should only do the python boxing/unboxing when I am really getting to Python. So I should try to define a boxing/unboxing and see if it is called when calling sub jitted functions.

In [7]:
import llvmlite
import numba

llvm_type = llvmlite.ir.IntType(64)


class TestStructType(numba.types.Type):
    def __init__(self):
        super().__init__(name="TestStruct")


test_struct_type = TestStructType()


def as_ptr(builder, value):
    new = builder.alloca(llvm_type)
    builder.store(value, new)
    return new
        

@numba.extending.register_model(TestStructType)
class TestStructModel(numba.extending.models.PrimitiveModel):
    def __init__(self, dmm, fe_type):
        super().__init__(dmm, fe_type, llvmlite.ir.PointerType(llvm_type))

    def get_return_type(self):
        return llvm_type

#     def get_data_type(self):
#         return ptr_llvm_type

#     def as_argument(self, builder, value):
#         return as_ptr(builder, value)

    def as_return(self, builder, value):
        return builder.load(value)

#     def as_data(self, builder, value):
#         return as_ptr(builder, value)

#     def from_argument(self, builder, value):
#         return builder.load(value)

    def from_return(self, builder, value):
        return as_ptr(builder, value)


@numba.extending.intrinsic
def create_test_struct(typingctx, int_t):

    sig = test_struct_type(int_t)

    def codegen(context, builder, sig, args):
        a = builder.alloca(llvm_type)
        builder.store(value=args[0], ptr=a)
        return a

    return sig, codegen


@numba.extending.intrinsic
def convert_test_struct(typingctx, test_struct_t):

    sig = numba.types.int64(test_struct_t)

    def codegen(context, builder, sig, args):
        return builder.load(args[0])

    return sig, codegen


@numba.njit
def inner_fn_works():
    a = create_test_struct(10)
    print("inner_fn_works", convert_test_struct(a))
    return a


@numba.njit
def outer_fn_works():
    print("outer_fn_works", convert_test_struct(inner_fn_works()))


@numba.njit
def inner_fn_broken():
    a = create_test_struct(10)
    return a


@numba.njit
def outer_fn_broken():
    print("outer_fn_broken", convert_test_struct(inner_fn_broken()))


outer_fn_works()
outer_fn_broken()


@numba.extending.box(TestStructType)
def box_test_struct(typ, val, c):
#     ret = c.builder.bitcast(val, ptr(char))
#     c.pyapi.incref(ret)
    return c.pyapi.bool_from_long(c.builder.load(val))


@numba.njit
def try_to_python():
    return create_test_struct(10)

print(try_to_python())

inner_fn_works 10
outer_fn_works 10
outer_fn_broken 10
i64*
True


Mmmk this seems to work. So it is only used when really going to Python, not when going between numba Njit functions.